In [1]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Mounted at /content/drive


# 신경망 학습

## 신경망 학습 과정 

1. 입력된 데이터가 각 층을 통과하며, 가중합(weight, bias) 및 활성화 함수 연산 반복
2. 출력층에서 계산된 값 출력
3. 예측값과 실제값(from 손실 함수)의 차이 계산
4. 경사하강법, 역전파를 통한 가중치 갱신
5. 학습 종료 까지 1~4 반복

## iteration

+ 가중치가 갱신되는 횟수.
+ 신경망 학습과정 1~4 반복 횟수(순전파 : 1,2 / 손실 함수 : 3 / 역전파 : 4)

## 순전파 (Forward Propagation)
입력층에서 입력된 신호가 각 층의 연산을 거쳐, 출력층에서 값으로 나오는 과정

1. 입력층(or 이전 은닉층)의 신호 수령
2. 가중합(가중치, 편향) 연산
3. 가중합 결과가 활성화 함수를 통과해 다음 층으로 전달

## 손실 함수 (Loss Function)
출력값과 타겟값의 차이를 구하는 함수
### MSE Mean-Squared Error
회귀 문제에 사용
### MAE Mean-Average Error
회귀 문제에 사용
### binary_crossentropy
이진 분류 문제에 사용
### CEE Cross-Entropy Error
다중 분류 문제에 사용
#### categorical_crossentropy
클래스간 비율이 유사 할 때 사용
#### sparse_categorical_crossentropy
클래스간 비율이 불균형 할 때 사용

## 역전파 (Backward Propagation)
출력층에서 입력층 방향으로, 경사하강법을 사용해, 손실을 줄이도록 weights와 bias를 갱신하는 과정. 이 과정에서 편미분과 Chain rule(연쇄 법칙)이 사용됨.

## 경사하강법 (Gradient Descent)
+ 손실 함수를 줄이는 방향으로
+ 손실 함수의 기울기가 작아지는 방향으로
+ 각 가중치 및 편향에 대한 손실 함수의 도함수(손실 함수를 편미분한 함수)를 계산해
+ 가중치 및 편향을 갱신하는 방법
+ 갱신된 가중치 = 갱신 전 가중치 - (학습률 * 해당 지점의 기울기)

## 옵티마이저 (Optimizer)
+ 경사를 내려가는 방법
+ 대표적인 것들은 아래와 같음
### GD (Gradient Descent, 경사하강법)
+ 일반적인 경사하강법
+ 모든 입력데이터를 사용해 가중치를 업데이트
### SGD (Stochastic Gradient Descent, 확률적 경사하강법)
+ 데이터의 양이 많아지면서, 기존의 GD는 학습 소요 시간이 지나치게 길어지면서 등장
+ 하나의 입력데이터만 사용해 가중치를 업데이트
+ 사용하는 데이터가 감소한 만큼, 소요 시간이 짧음
+ 이상치에 민감하게 작동함(불안정)
+ 1 epoch 에 하나의 데이터만 사용(다른 것들은 1 epoch에서 모든 데이터를 사용)
### 미니 배치 경사 하강법
+ SGD를 변형해 사용
+ N개의 데이터를 사용해 가중치를 업데이트
+ 속도도 개선되면서, 학습 안정성도 어느정도 유지
#### 배치 사이즈(batch size)
+ 미니 배치의 크기.
+ 일반적으로 2의 배수로 설정
+ 메모리가 허락하는 선에서 큰 사이즈가 학습에 안정적
### adam
+ 가장 안정적이고, 대중적인 옵티마이저

# 신경망 구현하기

In [2]:
# 라이브러리 임포트
import numpy as np
# 커스텀 라이브러리 임포트
from func_debug_log import debug

In [3]:
# debug_flag 설정(0:미출력, 1:출력)
debug_flag = 0

## 신경망 초기 상태 설정
weight, bias 초기화(랜덤값)

In [4]:
# 신경망 초기화 함수
#   params
#       없음
#   process
#       weights, bias이 랜덤하게 초기화된 net 생성
#   returns
#       초기화된 net (딕셔너리)
def init_network() :
    if debug_flag == 1 : debug('신경망 초기화', __name__)
    np.random.seed(42)
    net = {} 
    net['w1'] = np.random.rand(3, 4)
    net['b1'] = np.random.rand(4)
    net['w2'] = np.random.rand(4,2)
    net['b2'] = np.random.rand(2)
    return net

## 활성화 함수 구현
입력값의 시그모이드 함수의 결과값 

In [5]:
# 시그모이드 함수
#   params
#       x : 이전 층으로부터 전달 받은 가중합
#   process
#       x를 시그모이드 함수에 입력
#   returns
#       x가 시그모이드를 통과한 값
def sigmoid(x):
    if debug_flag == 1 : debug('시그모이드 함수 실행', __name__)
    z = 1/(1+np.exp(-x))
    return z

## 순전파
weight, bias 의 가중합 / 활성화 함수를 사용하여 입력층에서 은닉층을 거쳐, 출력층까지 계산.

### 입력층
데이터셋이 입력되는 층으로 features 수 만큼의 입력 노드를 갖는다.

### 은닉층
입력 신호를 가중합(가중치, 편향)하여 처리하는 층으로, 사용자가 볼 수 없으며, 노드수는 자유롭게 결정 가능하다. 2개 이상의 은닉층을 가진 신경망을 `딥러닝`이라고 한다.

### 출력층
연산을 마친 값이 출력되는 층이다. 문제의 종류에 따라, 노드 수, 활성화 함수가 결정된다.

#### 이진 분류
활성화 함수는 시그모이드, 노드 수 1개, 0~1 사이의 확률값이 출력

#### 다중 분류
활성화 함수는 소프트맥스, 노드 수는 레이블릐 클래스 수와 동일. 각 클래스일 확률을 0~1 사이 값으로 출력하며, 합은 1이 된다.

#### 회귀
활성화 함수는 사용하지 않으며, 출력층의 노드 수는 출력값의 특성 수와 동일.

In [10]:
# 순전파 함수
#   params
#       net : 학습시킬 신경망
#       x : 입력층에서 입력 받은 값
#   process
#       x를 은닉층(가중합) -> 시그모이드 함수 -> 출력층(가중합) 통과
#   returns
#       입력 x가 신경망을 통과해 산출된 값
def forward(net, x):
    if debug_flag == 1 : debug('순전파 함수 실행', __name__)
    w1 = net['w1']
    w2 = net['w2']
    b1 = net['b1']
    b2 = net['b2']

    # 은닉층
    h = np.dot(x, w1) + b1

    # 시그모이드 함수
    s = sigmoid(h)

    # 출력층
    o = np.dot(s, w2) + b2

    return o

In [7]:
if debug_flag == 1 : debug('신경망 인스턴스 생성', __name__)
net = init_network()

In [8]:
if debug_flag == 1 : debug('샘플 데이터 생성', __name__)
x = np.random.rand(3)

In [11]:
if debug_flag == 1 : debug('샘플데이터로, 신경망 순전파 진행', __name__)
y = forward(net, x)
print(y)

[1.59720489 1.78032042]
